In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import InjectedToolArg
from langchain_core.tools import tool
from dotenv import load_dotenv
import requests


In [34]:
load_dotenv()

True

In [35]:
api_key = "91b32d2b034022793a835278"
import json
from typing import Annotated

## Defining the tools for currency conversion and latest conversion factor

In [36]:
@tool
def get_conversion_factor(from_currency: str, to_currency: str) -> str:
    """
    Get the conversion factor between two currencies.
    """
    url = f"https://v6.exchangerate-api.com/v6/91b32d2b034022793a835278/pair/{from_currency}/{to_currency}"
    response = requests.get(url)
    
    if response.status_code != 200:
        raise ValueError(f"Error fetching conversion rate: {response.text}")
    
    data = json.loads(response.text)
    return data['conversion_rate']

@tool
def convert_currency(amount: int, coversion_factor: Annotated[float, InjectedToolArg]) -> float:
    """
    Convert an amount of money using the provided conversion factor.
    """
    return amount * float(coversion_factor)

In [37]:
get_conversion_factor.invoke({'from_currency': 'USD', 'to_currency': 'INR'})

85.4333

In [38]:
convert_currency.invoke({'amount': 100, 'coversion_factor': 82.5})

8250.0

## Initialize the LLM and messages

In [39]:
llm = ChatOpenAI()

## Bind the LLM with tools

In [40]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert_currency])

In [41]:
from langchain_core.messages import HumanMessage
messages = []

In [42]:
messages.append(HumanMessage(
    content="How much is 100 USD in INR and what is the conversion rate?"
))

ai_message = llm_with_tools.invoke(messages)


In [43]:
messages.append(ai_message)

In [44]:
messages

[HumanMessage(content='How much is 100 USD in INR and what is the conversion rate?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_38XVHJqJBjwmxpMCHitS2cd3', 'function': {'arguments': '{"from_currency": "USD", "to_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_ZZuGEmAW3iD3hZJ18L1nRhrQ', 'function': {'arguments': '{"amount": 100}', 'name': 'convert_currency'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 95, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BcX9Q7hYK4af7COVFHlz3jMiAQb9t', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs'

In [45]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'from_currency': 'USD', 'to_currency': 'INR'},
  'id': 'call_38XVHJqJBjwmxpMCHitS2cd3',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'amount': 100},
  'id': 'call_ZZuGEmAW3iD3hZJ18L1nRhrQ',
  'type': 'tool_call'}]

In [53]:
for tool_call in ai_message.tool_calls:
    if tool_call['name'] == "get_conversion_factor":
        conversion_factor = get_conversion_factor.invoke(tool_call)
        print(f"Conversion factor: {conversion_factor}")
        messages.append(conversion_factor)
    if tool_call['name'] == "convert_currency":
        tool_call['args']['coversion_factor'] = conversion_factor.content
        converted_amount = convert_currency.invoke(tool_call)
        print(f"Converted amount: {converted_amount}")
        messages.append(converted_amount)

Conversion factor: content='85.4333' name='get_conversion_factor' tool_call_id='call_38XVHJqJBjwmxpMCHitS2cd3'
Converted amount: content='8543.33' name='convert_currency' tool_call_id='call_ZZuGEmAW3iD3hZJ18L1nRhrQ'


In [55]:
llm_with_tools.invoke(messages).content

'100 USD is equivalent to 8543.33 INR and the conversion rate is 1 USD to 85.4333 INR.'